# Markowitz Efficient Frontier

## 1. Imports

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from tqdm import tqdm
import scipy.optimize

from fetchData import fetch_raw_data_yf, get_matrices, getNasdaqStocks
from MonteCarloRBA import MonteCarloRBA
from LearningRBA import MLRBA, find_correlation_matrix, maximize_sharpe, find_best_asset_to_remove


## 2. Fetch Data

### Get all Nasdaq Stocks

In [2]:
assets= [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "AMZN",  # Amazon.com Inc.
    "GOOGL", # Alphabet Inc. (Google) Class A
    "GOOG",  # Alphabet Inc. (Google) Class C
    "META",    # Meta Platforms Inc (formerly Facebook)
    "TSLA",  # Tesla Inc
    "UA", # Berkshire Hathaway Inc. Class B
    "JPM",   # JPMorgan Chase & Co.
    "V",     # Visa Inc.
    "JNJ",   # Johnson & Johnson
    "WMT",   # Walmart Inc.
    "PG",    # Procter & Gamble Co.
    "UNH",   # UnitedHealth Group Inc.
    "MA",    # Mastercard Inc.
    "NVDA",  # NVIDIA Corporation
    "HD",    # Home Depot Inc.
    "BAC",   # Bank of America Corp
    "DIS",   # Walt Disney Co
    "PYPL",  # PayPal Holdings
    "VZ",    # Verizon Communications Inc.
    "ADBE",  # Adobe Inc.
    "CMCSA", # Comcast Corporation
    "NFLX",  # Netflix Inc.
    "KO",    # Coca-Cola Co
    "NKE",   # NIKE Inc.
    "PFE",   # Pfizer Inc.
    "MRK",   # Merck & Co., Inc.
    "PEP",   # PepsiCo, Inc.
    "T",     # AT&T Inc.
    "ABT",   # Abbott Laboratories
    "CRM",   # Salesforce.com Inc.
    "ORCL",  # Oracle Corporation
    "ABBV",  # AbbVie Inc.
    "CSCO",  # Cisco Systems, Inc.
    "INTC",  # Intel Corporation
    "TMO",   # Thermo Fisher Scientific Inc.
    "XOM",   # Exxon Mobil Corporation
    "ACN",   # Accenture plc
    "LLY",   # Eli Lilly and Company
    "COST",  # Costco Wholesale Corporation
    "MCD",   # McDonald's Corp
    "DHR",   # Danaher Corporation
    "MDT",   # Medtronic plc
    "NEE",   # NextEra Energy, Inc.
    "BMY",   # Bristol-Myers Squibb Company
    "QCOM",  # Qualcomm Inc
    "CVX",   # Chevron Corporation
    "WFC",   # Wells Fargo & Co
    "LMT",    # Lockheed Martin Corporation
    "GS",   # Goldman Sachs Group, Inc.
    "MS",   # Morgan Stanley
    "IBM",  # International Business Machines Corporation
    "GE",   # General Electric Company
    "F",    # Ford Motor Company
    "GM",   # General Motors Company
    "UBER", # Uber Technologies, Inc.
    "LYFT", # Lyft, Inc.
    "SNAP", # Snap Inc.
    "TWTR", # Twitter, Inc.
    "SPOT", # Spotify Technology S.A.
    "AMD",  # Advanced Micro Devices, Inc.
    "TXN",  # Texas Instruments Incorporated
    "BABA", # Alibaba Group Holding Limited
    "SAP",  # SAP SE
    "HON",  # Honeywell International Inc.
    "BA",   # Boeing Company
    "RTX",  # Raytheon Technologies Corporation
    "CAT",  # Caterpillar Inc.
    "DE",   # Deere & Company
    "MMM",  # 3M Company
    "DUK",  # Duke Energy Corporation
    "SO",   # Southern Company
    "EXC",  # Exelon Corporation
    "NEE",  # NextEra Energy, Inc.
    "AEP",  # American Electric Power Company, Inc.
    "SRE",  # Sempra Energy
    "ETN",  # Eaton Corporation plc
    "EMR",  # Emerson Electric Co.
    "SYY",  # Sysco Corporation
    "KR",   # Kroger Co.
    "GIS",  # General Mills, Inc.
    "K",    # Kellogg Company
    "CPB",  # Campbell Soup Company
    "MO",   # Altria Group, Inc.
    "PM",   # Philip Morris International Inc.
    "BTI",  # British American Tobacco plc
    "RDY",  # Dr. Reddy's Laboratories Ltd.
    "GILD", # Gilead Sciences, Inc.
    "BIIB", # Biogen Inc.
    "CELG", # Celgene Corporation
    "AMGN", # Amgen Inc.
    "SYK",  # Stryker Corporation
    "BSX",  # Boston Scientific Corporation
    "ISRG", # Intuitive Surgical, Inc.
    "ZBH",  # Zimmer Biomet Holdings, Inc.
    "EW",   # Edwards Lifesciences Corporation
    "RMD",  # ResMed Inc.
    "VRTX", # Vertex Pharmaceuticals Incorporated
    "REGN",  # Regeneron Pharmaceuticals, Inc.
]

len(assets)

100

In [3]:
raw_data, asset_errors, max_combination= fetch_raw_data_yf(assets)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  99 of 99 completed

5 Failed downloads:
['SPOT', 'UBER', 'LYFT']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2018-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420088400, endDate = 1514782800")')
['CELG', 'TWTR']: YFTzMissingError('possibly delisted; no timezone found')


Omitted assets ( 5 ):  ['SPOT', 'CELG', 'UBER', 'LYFT', 'TWTR']
Time to fetch data: 3.65 seconds


## 3. Mean, Volatility and Covariance

In [4]:
names, cov, annualized_returns = get_matrices(raw_data, max_combination, None)[0]
volatility = np.sqrt(np.diag(cov))

risk_free_rate=0 
sharpe_ratios = (annualized_returns - risk_free_rate) / volatility

In [5]:
hover_texts = [
    f"{ticker} <br>Volatility: {vol:.2f} <br>Returns: {ret:.2%} <br>Sharpe Ratio: {sr:.2f}"
    for ticker, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
]

fig = go.Figure(data=go.Scatter(
    x=volatility, 
    y=annualized_returns, 
    mode='markers',
    hoverinfo='text',
    hovertext=hover_texts,
    marker=dict(color=sharpe_ratios, colorscale = 'RdBu', size=6, line=dict(width=1), colorbar=dict(title="Sharpe<br>Ratio")
    )
))

fig.update_layout(
    title='Markowitz Mean Varience Model',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Returns',
)

fig.show()


In [6]:
all_portfolios, dominant_portfolios = MonteCarloRBA(names, cov, annualized_returns, 1000)

100%|██████████| 1000/1000 [00:02<00:00, 369.39it/s]


In [7]:
print (len(dominant_portfolios) ,len(all_portfolios))

52 1000


In [8]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in all_portfolios],
    y=[p["return"] for p in all_portfolios],
    mode='markers',
    marker=dict(
        color=[p["return"] / (p["variance"]**0.5) for p in all_portfolios],
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.2%}<br>Volatility: {p['variance']**0.5:.2f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.2f}" for i in range(len(p['tickers']))])
        for p in all_portfolios
    ]
))

fig1.update_layout(
    xaxis=dict(title='Volatility (Standard Deviation)'),
    yaxis=dict(title='Annualised Returns'),
    title='Sample of Random Portfolios'
)

fig1.show()

In [9]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in dominant_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in dominant_portfolios],
    mode='markers',
    marker=dict(
        color=[p["return"] / (p["variance"]**0.5) for p in dominant_portfolios],  # Sharpe Ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.2%}<br>Volatility: {p['variance']**0.5:.2f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.2f}" for i in range(len(p['tickers']))])
        for p in dominant_portfolios
    ],
    name="Portfolios"
))


fig2.add_trace(go.Scatter(
    x=volatility, 
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=[
        f"{name} <br>Volatility: {vol:.2f} <br>Returns: {ret:.2%} <br>Sharpe Ratio: {sr:.2f}"
        for name, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
    ],
    marker=dict(
        color='brown',
        size=5,
        symbol='triangle-up',  # Sets the marker shape to a triangle
        line=dict(width=1)
    ),
    name="Individual Assets"
))

fig2.update_layout(
    title='Sample of Random Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5
    )
)


fig2.show()

In [ ]:
import numpy as np

def MLRBA(ticker, covariances, returns, num_iterations=10000):
    num_assets = 5
    portfolio = np.random.choice(list(ticker), num_assets, replace=False)
    print(ticker)
    curr_sharpe_ratio = -np.inf

    for _ in tqdm(range(num_iterations)):
        # Asset calculations
        curr_returns = returns.loc[portfolio].values
        curr_variances = covariances.loc[portfolio, portfolio].values
        asset_weights = maximize_sharpe(curr_returns, curr_variances)

        # Portfolio Calculations
        curr_portfolio_returns = np.dot(asset_weights, curr_returns)
        curr_portfolio_var = np.dot(asset_weights, curr_variances @ asset_weights)
        most_correlated_asset, _, _ = find_best_asset_to_remove(portfolio, curr_variances, curr_returns)
        #most_correlated_asset, _, _ = find_correlation_matrix(portfolio, curr_variances)

        # Remove the worst asset from the portfolio
        portfolio = [str(asset) for asset in portfolio if asset != most_correlated_asset]

        # Find the least correlated asset within all remaining assets (optional)
        # Substitute in and measure portfolio performance based on sharpe ratio
        # If Sharpe ratio was worse, then move on to the next least correlated asset

        # If Sharpe ratio is better, set as new base portfolio, and repeat the process for num_iterations times

    return portfolio, most_correlated_asset

portfolio = MLRBA(names, cov, annualized_returns, 1)

('AAPL_Close', 'V_Close', 'CSCO_Close', 'XOM_Close', 'TXN_Close', 'RMD_Close', 'GOOG_Close', 'EMR_Close', 'BIIB_Close', 'NFLX_Close', 'SYY_Close', 'IBM_Close', 'CPB_Close', 'ORCL_Close', 'AMGN_Close', 'BMY_Close', 'GM_Close', 'META_Close', 'BABA_Close', 'MS_Close', 'WMT_Close', 'BSX_Close', 'ABBV_Close', 'AMZN_Close', 'CMCSA_Close', 'WFC_Close', 'NEE_Close', 'PG_Close', 'MA_Close', 'GE_Close', 'RTX_Close', 'ACN_Close', 'JNJ_Close', 'PEP_Close', 'F_Close', 'ABT_Close', 'KO_Close', 'INTC_Close', 'T_Close', 'MRK_Close', 'DHR_Close', 'VZ_Close', 'CRM_Close', 'BA_Close', 'ADBE_Close', 'SAP_Close', 'CAT_Close', 'VRTX_Close', 'REGN_Close', 'PYPL_Close', 'MO_Close', 'ISRG_Close', 'GOOGL_Close', 'MSFT_Close', 'UA_Close', 'NVDA_Close', 'KR_Close', 'EW_Close', 'GIS_Close', 'AEP_Close', 'CVX_Close', 'SNAP_Close', 'EXC_Close', 'LLY_Close', 'PM_Close', 'MDT_Close', 'SYK_Close', 'UNH_Close', 'GS_Close', 'TSLA_Close', 'MCD_Close', 'COST_Close', 'BAC_Close', 'TMO_Close', 'ZBH_Close', 'DIS_Close', 'ETN_

100%|██████████| 1/1 [00:00<00:00, 221.97it/s]

['VRTX_Close' 'AEP_Close' 'K_Close' 'SYK_Close' 'WFC_Close']
K_Close
['VRTX_Close', 'AEP_Close', 'SYK_Close', 'WFC_Close']
